In [1]:
import pandas as pd

Edit this cell to read the file wanted. NOTE: will only work for csv files in the form of patient, features, outcome. 

Add code to use GPU

In [3]:
filename = "selected_features_stat.csv" # Add filename
filename2 = "selected_features_dyn.csv"
static = pd.read_csv(filename) # Load in Data
dynamic = pd.read_csv(filename2) # Load in Data

Run the following cell for Random Forest Model:

In [7]:
import numpy as np
%run RF.py 
print("static---------------------------------------------------------------------------------------------------------------")
actual, predicted, mean_feature_importances, X = RF(static)
rmse = np.sqrt(np.mean(predicted))
print("rmse: ", rmse)
print("Dynamic-------------------------------------------------------------------------------------------------------------")
actual, predicted, mean_feature_importances, X = RF(dynamic)
rmse = np.sqrt(np.mean(actual - predicted))
print("rmse: ", rmse)
# plots(data)

static---------------------------------------------------------------------------------------------------------------
Training Fold: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 
---------------- Random Forest ----------------
Root Mean Squared Error:  14.24634288958834
Feature Importance Values: , 0.01921874507454735, 0.150959551038571, 0.11776282116662291, 0.0505386129846102, 0.19901563722343082, 0.15411171419955505, 0.036035009405872585, 0.05363242968774373, 0.1456828125214079, 0.07304266669763851
Actual: , 38.4, 17.4667, 37.2, 16.8667, 36.2667, 14.6667, 51.0333, 6.6667, 54.3667, 30.7, 49.8667, 36.8, 15.1, 8.4333, 4.8667, 3.6, 32.3667, 7.4667, 33.9667, 43.1667, 31.5, 46.7333, 52.5, 36.2333, 17.6333, 7.7667, 20.4333, 11.2, 12.4, 44.6, 13.1, 5.5667, 19.3333, 4.9
Predct: , 31.5266, 15.3682, 17.4519, 22.159, 15.626, 31.8684, 31.8368, 17.0513, 28.014, 34.0527, 32.0441, 31.4356, 25.6044, 19.8721, 34.5752, 15.8157, 30.581, 28.1419, 18.5031, 2

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\ryanm\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ryanm\AppData\Local\Programs\Python\Python310\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Users\ryanm\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\ryanm\AppData\Local\Programs\Python\Python310\lib\asyncio\selector_events.py", line 115, in _read_from_se

Run the following cell for LightGBM. Add the number of trials you want optuna to run to tune parameters. 

In [ ]:
%run LightGBM.py 
# add your number of trails for optuna. longer trials -> better parameters, but longer runtime
# n_trails = 150
# lightGBMLOOOptuna(data, n_trails)

params = ParamStatic()
# params = ParamDyn()
mean_feature_importances, X, actual, predicted = lightGBMLOO(data, params)

# plots(data, params)

Run the follwing cell for XGBoost. 

In [ ]:
%run XGBoost.py 
n = 150 # edit the number of trials
lightGBMLOOOptuna(data, n)

Run the following cell for MultiLinearRegression. With no Feature Importance function built into the model I excluded that graph. However, it is possible to create it in the future, if needed. 

In [ ]:
%run MultiLinearRegression.py 
cof_list, actual, predicted, X= MLR(data) # will run the model and give you the RMSQ
# plot(data) # will run the model and plot results
